# Column Generation for the Cutting Stock Problem

## Objectives and Prerequisites

In this notebook, you will:

1. Learn about the Cutting Stock Problem, a classical problem from the Integer Programming (IP) literature.
2. Discover how delayed column generation can be implemented in Gurobi to solve this problem more efficiently.

This modelling example is at the advanced level. To fully understand the content of this notebook, the reader should be familiar with the following:

- Object-Oriented Programming.
- The Knapsack Problem.
- Delayed column generation as a decomposition technique.

-----

## Motivation

Despite the great advancements that we've seen in mathematical optimization solver technology, some problems are still challenging to solve due to their large number of variables or constraints. Decomposition techniques — such as delayed column generation or Benders decomposition — apply the principle of "divide and conquer" to tackle these challenging problems.

Delayed column generation is typically used with large problems, which have so many variables that it's not possible to consider all of them explicitly. Experience with large problems indicates that, usually, most of these variables never enter the basis. Therefore, only a subset of these variables need to ever be considered when solving the problem. Delayed column generation exploits this fact to only generate variables (i.e. columns) that have the potential to improve the objective function. It's interesting to note that generating a column in the primal formulation amounts to adding a constraint to its dual. Thus, you can think of this technique as a cutting plane method in the dual space.

Delayed column generation has been successfully applied to many problems such as crew scheduling, vehicle routing, the capacitated p-median problem, and the Cutting Stock Problem. We will use the latter to illustrate how this technique can be implemented in the Gurobi-Python interface.

-----

## Problem Description

The Cutting Stock Problem deals with the problem of cutting stock material with the same, fixed width — such as paper rolls — into smaller pieces, according to a set of orders specifying both the widths and the demand requirements, so as to minimize the amount of wasted material.

![image](./csp_ex.png)

-----

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

- Sets and indices.
- Parameters.
- Decision variables.
- Objective function(s).
- Constraints.

First, we present a naïve IP that doesn't scale well because its linear relaxation is not tight. Then, we introduce a Full Master Problem that explicitly enumerates all possible cut patterns, which turns out to be amenable to delayed column generation.

### Naïve IP

#### Sets and Indices

$i \in \text{Stock}=\{1,2,\dots, N\}$: Set of available stock material units.

$j \in \text{Pieces}$: Set of final pieces.

#### Parameters

$N \in \mathbb{N}$: Upper bound for the number of stock material units that can be used.

$\text{Stock_length} \in \mathbb{R}^+$: Length of any stock material unit.

$\text{Piece_length}_j \in \mathbb{R}^+$: Length of final piece $j$.

$\text{Requirement}_j \in \mathbb{N}$: Demand requirement of final piece $j$.

#### Decision Variables

$\text{use}_i \in \{0,1\}$: 1 if stock material unit $i$ is used; 0 otherwise.

$\text{cut}_{i,j} \in \mathbb{N}$: Number of pieces of type $j$ cut out from stock material unit $i$.

#### Objective Function

- **Material Consumption:** Minimize the number of stock material units used.

$$
\begin{equation}
\text{Min} \sum_{i \in \text{Stock}}{\text{use}_i}
\tag{1.0}
\end{equation}
$$

#### Constraints

- **Satisfy Demand:** Total number of final pieces of type $j$ obtained must satisfy its demand requirement.

$$
\begin{equation}
\sum_{i \in \text{Stock}}{\text{cut}_{i,j}} \geq \text{Requirement}_j \quad \forall j \in \text{Pieces}
\tag{1.1}
\end{equation}
$$

- **Capacity:** Total length of the pieces cut out from stock material unit $i$ cannot exceed its length.

$$
\begin{equation}
\sum_{j \in \text{Pieces}}{\text{Piece_length}_j \cdot \text{cut}_{i,j}} \leq \text{Stock_length} \cdot \text{use}_i \quad \forall i \in \text{Stock}
\tag{1.2}
\end{equation}
$$

- **Symmetry breaking:** Stock material unit $i+1$ cannot be used if the previous unit $i$ wasn't. Note that this constraint set is not needed for correctness, but certainly helps to reduce the search space.

$$
\begin{equation}
\text{use}_{i+1} \leq \text{use}_i \quad \forall i \in \text{Stock} \setminus \{N\}
\tag{1.3}
\end{equation}
$$

### Full Master Problem

Now, instead of focusing on which stock material unit a particular piece is to be cut from, we consider all possible patterns (potentially generating multiple pieces of different widths) that can be used to cut from a stock material unit. The task is now to find how many times a particular pattern should be used.

#### Sets and Indices

$p \in \text{Patterns}$: Set of all possible cutting patterns, whose size grows exponentially with the number of final pieces. Note that each of these patterns implicitly considers the lengths of the final pieces and the stock material units, which in fact can be viewed as a knapsack constraint.

$j \in \text{Pieces}$: Set of final pieces.

#### Parameters

$\text{Requirement}_j \in \mathbb{N}$: Demand requirement of final piece $j$.

$\text{Frequency}_{p,j} \in \mathbb{N}$: Number of pieces of type $j$ cut out from pattern $p$.

#### Decision Variables

$\text{count}_p \in \mathbb{N}$: Number of times pattern $p$ is used.

#### Objective Function

- **Material Consumption:** Minimize the number of stock material units used.

$$
\begin{equation}
\text{Min} \sum_{p \in \text{Patterns}}{\text{count}_p}
\tag{2.0}
\end{equation}
$$

#### Constraints

- **Satisfy Demand:** Total number of final pieces of type $j$ obtained must satisfy its demand requirement.

$$
\begin{equation}
\sum_{p \in \text{Patterns}}{\text{Frequency}_{p,j} \cdot \text{count}_p} \geq \text{Requirement}_j \quad \forall j \in \text{Pieces}
\tag{2.1}
\end{equation}
$$

Recall that the number of patterns grows exponentially with the number of final pieces, so in practice the Full Master Problem cannot be solved in one shot. Instead, we consider a Restricted Master Problem (RMP), initialized with a few trivial patterns. Then we follow an iterative approach, alternating between solving a knapsack problem (i.e. the subproblem) to generate a new pattern (i.e. the entering column) and solving a larger RMP to update the objective function — representing the reduced cost of the next pattern to be selected — of the subproblem. This cycle is repeated until the reduced cost of the entering column is non-negative.

-----

## Python Implementation

In the following implementation of column generation, we use two main libraries:

- Numpy for scientific computing.
- Gurobi for mathematical optimization.

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB


class MasterProblem:
    def __init__(self):
        self.model = gp.Model("master")
        self.vars = None
        self.constrs = None
        
    def setup(self, patterns, demand):
        num_patterns = len(patterns)
        self.vars = self.model.addVars(num_patterns, obj=1, name="Pattern")
        self.constrs = self.model.addConstrs((gp.quicksum(patterns[pattern][piece]*self.vars[pattern]
                                                          for pattern in range(num_patterns))
                                              >= demand[piece] for piece in demand.keys()),
                                             name="Demand")
        self.model.modelSense = GRB.MINIMIZE
        # Turning off output because of the iterative procedure
        self.model.params.outputFlag = 0
        self.model.update()
        
    def update(self, pattern, index):
        new_col = gp.Column(coeffs=pattern, constrs=self.constrs.values())
        self.vars[index] = self.model.addVar(obj=1, column=new_col,
                                             name=f"Pattern[{index}]")
        self.model.update()


class SubProblem:
    def __init__(self):
        self.model = gp.Model("subproblem")
        self.vars = {}
        self.constr = None
        
    def setup(self, stock_length, lengths, duals):
        self.vars = self.model.addVars(len(lengths), obj=duals, vtype=GRB.INTEGER,
                                       name="Frequency")
        self.constr = self.model.addConstr(self.vars.prod(lengths) <= stock_length,
                                           name="Knapsack")
        self.model.modelSense = GRB.MAXIMIZE
        # Turning off output because of the iterative procedure
        self.model.params.outputFlag = 0
        # Stop the subproblem routine as soon as the objective's best bound becomes
        #less than or equal to one, as this implies a non-negative reduced cost for
        #the entering column.
        self.model.params.bestBdStop = 1
        self.model.update()
        
    def update(self, duals):
        self.model.setAttr("obj", self.vars, duals)
        self.model.update()


class CuttingStock:
    def __init__(self, stock_length, pieces):
        self.stock_length = stock_length
        self.pieces, self.lengths, self.demand = gp.multidict(pieces)
        self.patterns = None
        self.duals = [0]*len(self.pieces)
        piece_reqs = [length*req for length, req in pieces.values()]
        self.min_rolls = np.ceil(np.sum(piece_reqs)/stock_length)
        self.solution = {}
        self.master = MasterProblem()
        self.subproblem = SubProblem()
        
    def _initialize_patterns(self):
        # Find trivial patterns that consider one final piece at a time,
        #fitting as many pieces as possible into the stock material unit
        patterns = []
        for idx, length in self.lengths.items():
            pattern = [0]*len(self.pieces)
            pattern[idx] = self.stock_length // length
            patterns.append(pattern)
        self.patterns = patterns
        
    def _generate_patterns(self):
        self._initialize_patterns()
        self.master.setup(self.patterns, self.demand)
        self.subproblem.setup(self.stock_length, self.lengths, self.duals)
        while True:
            self.master.model.optimize()
            self.duals = self.master.model.getAttr("pi", self.master.constrs)
            self.subproblem.update(self.duals)
            self.subproblem.model.optimize()
            reduced_cost = 1 - self.subproblem.model.objVal
            if reduced_cost >= 0:
                break
            
            pattern = [0]*len(self.pieces)
            for piece, var in self.subproblem.vars.items():
                if var.x > 0.5:
                    pattern[piece] = round(var.x)
            self.master.update(pattern, len(self.patterns))
            self.patterns.append(pattern)
    def solve(self):
        """
        Gurobi does not support branch-and-price, as this requires to add columns
        at local nodes of the search tree. A heuristic is used instead, where the
        integrality constraints for the variables of the final root LP relaxation
        are installed and the resulting MIP is solved. Note that the optimal
        solution could be overlooked, as additional columns are not generated at
        the local nodes of the search tree.
        """
        self._generate_patterns()
        self.master.model.setAttr("vType", self.master.vars, GRB.INTEGER)
        self.master.model.optimize()
        for pattern, var in self.master.vars.items():
            if var.x > 0.5:
                self.solution[pattern] = round(var.x)


### Test Instance

We now test our implementation using a toy example with 13 final pieces to be cut out from stock material units of 5,600 cm. The set of orders specifying the lengths and the demand requirements of final pieces is presented below:

| ID | Final Piece Length (cm) | Demand (units)|
|----|----|----|
| 0 | 1380 | 22 |
| 1 |  1520 | 25 |
| 2 | 1560 | 12 |
| 3 | 1710 | 14 |
| 4 | 1820 | 18 |
| 5 | 1880 | 18 |
| 6 | 1930 | 20 |
| 7 | 2000 | 10 |
| 8 | 2050 | 12 |
| 9 | 2100 | 14 |
| 10 | 2140 | 16 |
| 11 | 2150 | 18 |
| 12 | 2200 | 20 |

In [2]:
# Dictionary that maps each final piece ID with their width and demand requirement
orders = {0:[1380,22],
          1:[1520,25],
          2:[1560,12],
          3:[1710,14],
          4:[1820,18],
          5:[1880,18],
          6:[1930,20],
          7:[2000,10],
          8:[2050,12],
          9:[2100,14],
          10:[2140,16],
          11:[2150,18],
          12:[2200,20]}

mycsp = CuttingStock(5600, orders)
mycsp.solve()

Using license file c:\gurobi\gurobi.lic


The patterns generated are the following:

In [3]:
mycsp.patterns

[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

The patterns that are actually used and their respective frequency are reported below:

| Pattern ID | Frequency | Description |
|----|----|----|
| 4 | 1 | 3 units of final piece 4 |
| 17 | 7 | 1 unit and 2 units of final pieces 0 and 9 |
| 19 | 2 | 1 unit of final pieces 2, 5, and 10 |
| 20 | 4 | 2 unit and 1 units of final pieces 3 and 10 |
| 22 | 1 | 2 unit and 1 units of final pieces 3 and 11 | 
| 26 | 5 | 1 unit of final pieces 1, 5, and 10 |
| 28 | 17 | 1 unit of final pieces 1, 6, and 11 |
| 29 | 4 | 1 unit and 2 units of final pieces 4 and 5 |
| 31 | 10 | 1 unit of final pieces 0, 7, and 12 |
| 32 | 3 | 1 unit of final pieces 2, 6, and 8 |
| 34 | 4 | 1 unit of final pieces 3, 4, and 8 |
| 35 | 5 | 1 unit of final pieces 0, 8, and 10 |
| 36 | 3 | 1 unit of final pieces 1, 5, and 12 |
| 38 | 7 | 1 unit of final pieces 2, 4, and 12 |

In [4]:
print(mycsp.solution)

{4: 1, 17: 7, 19: 2, 20: 4, 22: 1, 26: 5, 28: 17, 29: 4, 31: 10, 32: 3, 34: 4, 35: 5, 36: 3, 38: 7}


The minimum number of stock material units that must be used is:

In [5]:
print(mycsp.min_rolls)

73.0


The actual number of stock material units used by the solution found is:

In [6]:
sum(mycsp.solution.values())

73

## Conclusion

This notebook showed how to implement delayed column generation using Gurobi in an object-oriented framework. It also suggested a heuristic that can be used in lieu of a branch-and-price scheme, in the event that this is not compatible with Gurobi. Finally, it's worth mentioning that the reader can improve the performance of this implementation by replacing the subproblem routine, which uses a MIP solver, with an algorithm specifically tailored for the Knapsack problem.

-----

## References

1. Desaulniers, G., Desrosiers, J., & Solomon, M. M. (Eds.). (2006). Column generation (Vol. 5). Springer Science & Business Media.
2. Kantorovich, L. V., & Zalgaller, V. A. (1951). Calculation of rational cutting of stock. Lenizdat, Leningrad, 5, 11-14.

Copyright © 2021 Gurobi Optimization, LLC